In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

Useful Functions!

In [ ]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MAP VISUALIZATION #############################################

# df_to_dash_table converts a DataFrame to a Dash HTML table for display in the popup of EU_DV_1

def df_to_dash_table(df):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(len(df))
        ])
    ], style={"maxWidth": "300px", "fontSize": "12px"})

Definition of variables for visualization

In [ ]:
# prov is a GeoDataFrame containing the geometries of the provinces
prov=get_province_shape()

# geojson is a dictionary containing the geometry of the provinces in GeoJSON format
geojson = mapping(prov.geometry_province)

# the two following variables are used to create a green and a red icon marker, later used in the map

#https://github.com/pointhi/leaflet-color-markers
custom_icon_green = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)
custom_icon_red = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)

# How to render geojson.
point_to_layer = assign("""function(feature, latlng, context){
    const p = feature.properties;
    if(p.type === 'circlemarker'){return L.circleMarker(latlng, radius=p._radius)}
    if(p.type === 'circle'){return L.circle(latlng, radius=p._mRadius)}
    return L.marker(latlng);
}""")


In [ ]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([

    html.H1('Bugs_project: Air quality analysis'),
    html.P('Description of the dashboard functionalities'),

    ############################################# User selection #############################################
    html.Div([
        html.H2('User selection'),
        html.P('Select the type of user you are:'),
        dcc.RadioItems(
            id='user-selection',
            options=[
                {'label': 'Normal User', 'value': 1},
                {'label': 'Expert User', 'value': 2}
            ],
            value=0,
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),
    ], id='user', hidden=False, style={'padding': 20, 'margin':2, 'border': '1px solid #ccc'}),

    ############################################# EU_DV_2 #############################################

        html.Div([
        html.H2('EU_DV_2 Location of all the sensors of a selected pollutant'),
        html.P('Choose a pollutant, and then visualize on a map the location af all the sensors of that pollutant. The sensor will appear red if it\'s still active, and will appear gray if it is not active anymore. Clicking on the icon of a sensor the user can visualiza the id of the sensor, the name of the station in which the sensor is located and date in which the sensor was deactivated, if it is not active anymore'),
        
        #Dropdown to select the pollutant
            dcc.Dropdown(
            id='all-pollutant-dropdown_EU_DV_2',
            options=all_pollutant_dropdown(),
            placeholder="Select a pollutant",
            style={'margin-top':20}
        ),

          

         # Dropdown per selezionare la provincia
            dcc.Dropdown(
            id='province-dropdown_EU_DV_2',
            options=all_province_dropdown(),  # Assicurati che questa funzione restituisca un elenco di opzioni
            placeholder="Select a province",
            style={'margin-top': 20}
        ),

    # Intervallo di selezione per le date (start - end)
    dcc.DatePickerRange(
        id='date-range-picker_EU_DV_2',
        min_date_allowed=date(2020, 1, 1),  # Imposta il minimo consentito
        max_date_allowed=date(2025, 12, 31),  # Imposta il massimo
        start_date_placeholder_text="Start Date",
        end_date_placeholder_text="End Date",
        display_format='YYYY-MM-DD',
        style={'margin-top': 20}
    )


        #Button to activate the histogram
        html.Div([
            html.Button('Visualize/update histogram', id='button_EU_DV_2', n_clicks=0, style={'margin-top': 20, 'margin-bottom':20, 'padding': '6px 12px', 'fontSize': '15px',})
        ], style={'textAlign': 'center'}),
        #Visualize the histogram
])

# Run the app in Jupyter notebook
print("Starting Dashboard...")
#app.run_server(port=8089)
app.run(port=8089, debug=True)